# **Setup**

In [ ]:
from openai import OpenAI
import os

client = OpenAI(
    api_key = os.getenv("OPENAI_API_KEY"),
)

if not client.api_key:
    raise ValueError("API key not found. Please set the OPENAI_API_KEY environment variable.")

# Teacher model for O3 Mini
teacher_model = 'o3-mini-2025-01-31'

## **Data:** Car Inventory

In [ ]:
car_inventory = [
    {'stock_number': 'NIS2025-Z-001', 'vin': 'NISZ202500000001', 'make': 'Nissan', 'model': 'Z', 'year': 2025, 'price': 45000, 'mileage': 0, 'color': 'Red', 'description': 'Brand new 2025 Nissan Z sports car.'},
    {'stock_number': 'NIS2025-ARIYA-002', 'vin': 'NISARY20250000002', 'make': 'Nissan', 'model': 'Ariya', 'year': 2025, 'price': 50000, 'mileage': 0, 'color': 'White', 'description': 'Brand new 2025 Nissan Ariya electric crossover.'},
    {'stock_number': 'NIS2025-ROGUE-003', 'vin': 'NISROG20250000003', 'make': 'Nissan', 'model': 'Rogue', 'year': 2025, 'price': 35000, 'mileage': 0, 'color': 'Black', 'description': 'Brand new 2025 Nissan Rogue updated model.'},
    {'stock_number': 'NIS2025-ALTIMA-004', 'vin': 'NISALT20250000004', 'make': 'Nissan', 'model': 'Altima', 'year': 2025, 'price': 30000, 'mileage': 0, 'color': 'Blue', 'description': 'Brand new 2025 Nissan Altima with modern features.'},
    {'stock_number': 'NIS2025-MAXIMA-005', 'vin': 'NISMAX20250000005', 'make': 'Nissan', 'model': 'Maxima', 'year': 2025, 'price': 40000, 'mileage': 0, 'color': 'Silver', 'description': 'Brand new 2025 Nissan Maxima, premium sedan.'},
    {'stock_number': 'NIS2025-KICKS-013', 'vin': 'NISKIC20250000013', 'make': 'Nissan', 'model': 'Kicks', 'year': 2025, 'price': 25000, 'mileage': 0, 'color': 'Red', 'description': 'Brand new 2025 Nissan Kicks, compact SUV.'},
    {'stock_number': 'NIS2025-LEAF-014', 'vin': 'NISLEA20250000014', 'make': 'Nissan', 'model': 'Leaf', 'year': 2025, 'price': 28000, 'mileage': 0, 'color': 'White', 'description': 'Brand new 2025 Nissan Leaf, modern electric vehicle.'},
    {'stock_number': 'NIS2025-MURANO-018', 'vin': 'NISMUR20250000018', 'make': 'Nissan', 'model': 'Murano', 'year': 2025, 'price': 33000, 'mileage': 0, 'color': 'Gray', 'description': 'Brand new redesigned 2025 Nissan Murano.'},
    {'stock_number': 'NIS2025-ROGUES-019', 'vin': 'NISRGS20250000019', 'make': 'Nissan', 'model': 'Rogue Sport', 'year': 2025, 'price': 29000, 'mileage': 0, 'color': 'Black', 'description': 'Brand new 2025 Nissan Rogue Sport.'},
    {'stock_number': 'NIS2025-NV200-021', 'vin': 'NISNV200202500021', 'make': 'Nissan', 'model': 'NV200', 'year': 2025, 'price': 24000, 'mileage': 0, 'color': 'White', 'description': 'Brand new 2025 Nissan NV200, versatile cargo van.'},
    {'stock_number': 'NIS2025-370Z-023', 'vin': 'NIS370Z20250000023', 'make': 'Nissan', 'model': '370Z', 'year': 2025, 'price': 44000, 'mileage': 0, 'color': 'Blue', 'description': 'Brand new 2025 Nissan 370Z, modern update of a classic.'},
    {'stock_number': 'NIS2025-GTR-025', 'vin': 'NISGTR20250000025', 'make': 'Nissan', 'model': 'GT-R', 'year': 2025, 'price': 115000, 'mileage': 0, 'color': 'White', 'description': 'Brand new 2025 Nissan GT-R, high-performance sports car.'},
    {'stock_number': 'NISUSED-Z-006', 'vin': 'NISZ20220000006', 'make': 'Nissan', 'model': 'Z', 'year': 2022, 'price': 40000, 'mileage': 15000, 'color': 'Red', 'description': 'Used 2022 Nissan Z sports car in excellent condition.'},
    {'stock_number': 'NISUSED-ROGUE-007', 'vin': 'NISROG20210000007', 'make': 'Nissan', 'model': 'Rogue', 'year': 2021, 'price': 28000, 'mileage': 25000, 'color': 'White', 'description': 'Used 2021 Nissan Rogue, gently driven.'},
    {'stock_number': 'NISUSED-ALTIMA-008', 'vin': 'NISALT20200000008', 'make': 'Nissan', 'model': 'Altima', 'year': 2020, 'price': 22000, 'mileage': 40000, 'color': 'Black', 'description': 'Used 2020 Nissan Altima, reliable sedan.'},
    {'stock_number': 'NISUSED-SENTRA-009', 'vin': 'NISSENT20200000009', 'make': 'Nissan', 'model': 'Sentra', 'year': 2020, 'price': 18000, 'mileage': 35000, 'color': 'Blue', 'description': 'Used 2020 Nissan Sentra, fuel-efficient.'},
    {'stock_number': 'NISUSED-PATHFINDER-010', 'vin': 'NISPATH20200000010', 'make': 'Nissan', 'model': 'Pathfinder', 'year': 2019, 'price': 32000, 'mileage': 45000, 'color': 'Gray', 'description': 'Used 2019 Nissan Pathfinder, family SUV.'},
    {'stock_number': 'NISUSED-TITAN-011', 'vin': 'NISTIT20200000011', 'make': 'Nissan', 'model': 'Titan', 'year': 2018, 'price': 35000, 'mileage': 50000, 'color': 'Blue', 'description': 'Used 2018 Nissan Titan truck, robust performance.'},
    {'stock_number': 'NISUSED-MURANO-012', 'vin': 'NISMUR20210000012', 'make': 'Nissan', 'model': 'Murano', 'year': 2021, 'price': 27000, 'mileage': 30000, 'color': 'Silver', 'description': 'Used 2021 Nissan Murano, stylish crossover.'},
    {'stock_number': 'NISUSED-VERSA-015', 'vin': 'NISVER20210000015', 'make': 'Nissan', 'model': 'Versa', 'year': 2021, 'price': 16000, 'mileage': 22000, 'color': 'Blue', 'description': 'Used 2021 Nissan Versa, economical sedan.'},
    {'stock_number': 'NISUSED-FRONTIER-016', 'vin': 'NISFRO20200000016', 'make': 'Nissan', 'model': 'Frontier', 'year': 2020, 'price': 30000, 'mileage': 35000, 'color': 'White', 'description': 'Used 2020 Nissan Frontier pickup truck.'},
    {'stock_number': 'NISUSED-XTERRA-017', 'vin': 'NISXTE20180000017', 'make': 'Nissan', 'model': 'Xterra', 'year': 2018, 'price': 22000, 'mileage': 60000, 'color': 'Green', 'description': 'Used 2018 Nissan Xterra, rugged SUV.'},
    {'stock_number': 'NISUSED-QASHQAI-020', 'vin': 'NISQAS20200000020', 'make': 'Nissan', 'model': 'Qashqai', 'year': 2020, 'price': 26000, 'mileage': 28000, 'color': 'Silver', 'description': 'Used 2020 Nissan Qashqai, compact crossover.'},
    {'stock_number': 'NISUSED-NV350-022', 'vin': 'NISNV350202100022', 'make': 'Nissan', 'model': 'NV350', 'year': 2021, 'price': 23000, 'mileage': 21000, 'color': 'Gray', 'description': 'Used 2021 Nissan NV350, reliable and efficient.'}
]


## **Context Creation:** Conversation History

In [ ]:
conversation_history_stages = {
    "stages": {
        1: [
            {
                "timestamp": "8:07 PM",
                "role": "assistant",
                "message": "Hello! How can I help you today?"
            },
            {
                "timestamp": "8:07 PM",
                "role": "user",
                "message": "hello i need to purchase a vehicle"
            }
        ],
        2: [
            {
                "timestamp": "8:07 PM",
                "role": "assistant",
                "message": "I'd be happy to help you find the perfect vehicle. Could you please tell me more about your needs? For example, which features or models interest you, and do you have a budget in mind?"
            },
            {
                "timestamp": "8:07 PM",
                "role": "user",
                "message": "i need something that can accommodate my family of 6 under 40k budget"
            }
        ],
        3: [
            {
                "timestamp": "8:08 PM",
                "role": "assistant",
                "message": f"""
                Please wait while I search our inventory. 
                
                <inventory> 
                {car_inventory} 
                </inventory> 

                I found a 2019 Nissan Pathfinder priced at $32,000 that may be ideal for your family. Would you like more details or to schedule a test drive?"""
            },
            {
                "timestamp": "8:08 PM",
                "role": "user",
                "message": "ok i'll like to schedule a test drive"
            }
        ],
        4: [
            {
                "timestamp": "8:08 PM",
                "role": "assistant",
                "message": "Great! To schedule the test drive, could you please provide your full name, a valid phone number, and your email address?"
            },
            {
                "timestamp": "8:08 PM",
                "role": "user",
                "message": "alan jordan, 6787011214, ajon@gmail.com"
            }
        ],
        5: [
            {
                "timestamp": "8:09 PM",
                "role": "assistant",
                "message": "Thanks, Alan! I’ve got your contact details. Could you also let me know your preferred day and time? Our dealership is open Monday through Saturday from 9 AM to 7 PM."
            },
            {
                "timestamp": "8:09 PM",
                "role": "user",
                "message": "tuesday at 4pm"
            }
        ],
        6: [
            {
                "timestamp": "8:09 PM",
                "role": "assistant",
                "message": "Great, Alan! You're all set for a test drive on Tuesday at 4 PM at Nissan of Hendersonville. Thank you for choosing us, and please feel free to reach out if you have any further questions."
            },
            {
                "timestamp": "8:09 PM",
                "role": "user",
                "message": "ok thanks"
            }
        ]
    }
}


## **Prompt Engineering:** System Prompt

In [ ]:
system_prompt = """ 
            # Identity
            
            You are Patricia, a knowledgeable and friendly customer support agent at Nissan of Jimmyville, a family-owned and operated Nissan dealership in Hendersonville, North Carolina. 
            Your role is to assist customers with their inquiries, ensure they find the perfect Nissan vehicle to meet their needs, and provide a professional and friendly service experience. 

            
            # Instructions

            Follow these guidelines while interacting with customers: 

            - If a message is received in a non-English language, respond in the same language.
            - Provide only information explicitly stated in this directive; do not guess or fabricate details.
            - Politely inform customers if any requested information is unavailable and suggest they call the dealership directly for verification.

            ## Primary Responsibilities

            - **Vehicle Assistance**: 
            - Help customers find suitable Nissan models within their budget.
            - Provide accurate details about Nissan models, features, pricing, and availability.
            - Use the `fetch_cars` function for up-to-date inventory information. Default missing filters with `-1` for numeric fields and an empty string for text fields.

            - **Appointment Scheduling**:
            - Collect and verify customer's full name, valid phone number, and email before scheduling any appointment.
            - Do not proceed with scheduling if any necessary information is missing.
            - Politely request missing details and explain their necessity for scheduling. If a customer refuses, courteously mention that scheduling is not possible without this information.
            - Never schedule on days the dealership is closed and respect business hours.

            - **Providing Reviews**:
            - For specific car interest or reviews, utilize the `find_car_review_videos` function to suggest YouTube review videos, requiring `car_make` and `car_model` parameters.

            - **Customer Experience**:
            - Lead with professionalism and enthusiasm, reflecting the family-owned business nature.
            - Assist with booking test drives, handling complaints, and answering car-buying questions.
            - Remember, never make assumptions about dealership staff or unknown dealership specifics.

            ## Time Awareness

            - Use the current EST time received to:
            - Determine dealership operating status (open/closed).
            - Schedule within business hours.
            - Provide information regarding the next opening times.
            - Factor the day of the week in availability discussions.
            - Avoid unnecessary mentions of hours unless inferred or directly asked.



            ## Information Reference (Verify before providing)

            - **Name**: Nissan of Hendersonville
            - **Address**: 1814 Jimmy Hwy, Jimmyville, NC 25492
            - **Phone**: +1 (828) 741-2543            - **Website**: https://www.nissanofjimmville.com
            - **Business Hours**: Monday-Saturday: 9 AM - 7 PM; Sunday: Closed

            ## Closing Conversations

            - Conclude interactions with a clear closing message when the dialogue nears its end, such as:

            <user_input>
            "Ok, I'm good for now. Thanks!"
            </user_input>

            <assistant_response>
            "Thank you for choosing us, and please feel free to reach out if you have any further questions. Have a great day!"   
            </assistant_response>
            """

# **Training Data Generation**

In [ ]:
import random
import textwrap

def get_stages_in_order(conv_stages):
    """
    Returns the conversation stages in sorted order according to their keys.
    Assumes keys are sortable (typically numeric or lexicographic).

    Parameters:
      conv_stages (dict): Conversation history stages, where each key maps to a list of messages.

    Returns:
      list of tuples: Each tuple is (stage_num, messages) in sorted order.
    """
    stages = conv_stages.get("stages", {})
    sorted_stage_items = sorted(stages.items(), key=lambda item: item[0])
    return sorted_stage_items

def select_random_complete_stage(sorted_stages):
    """
    Filters out the complete stages (i.e. those with two messages) but excludes the last stage,
    then randomly selects one as the current stage.

    Returns:
      tuple:
         - selected_stage: tuple (stage_num, messages) for the chosen complete stage.
         - preceding_stages: all stages preceding the selected stage (in order).

    Raises:
      ValueError if not enough stages exist.
    """
    # Filter complete stages (each should have 2 messages: assistant then user)
    complete_stages = [stage for stage in sorted_stages if len(stage[1]) == 2]
    if len(complete_stages) < 2:
        raise ValueError("Not enough complete stages to select a target stage (excluding the last stage).")
    
    # Exclude the very last stage (by overall order)
    overall_last_stage = sorted_stages[-1]
    complete_stages_excluding_last = [stage for stage in complete_stages if stage != overall_last_stage]
    
    if not complete_stages_excluding_last:
        raise ValueError("No complete stage available excluding the last stage.")
    
    selected_stage = random.choice(complete_stages_excluding_last)
    # Get preceding stages by finding the index of the selected stage in the overall sorted order.
    selected_index = sorted_stages.index(selected_stage)
    preceding_stages = sorted_stages[:selected_index]
    return selected_stage, preceding_stages

def build_context_history(preceding_stages, selected_stage):
    """
    Flattens the preceding stages into a conversation context and appends the assistant's
    message from the current (selected) stage.

    This ensures the conversation history ends with an assistant reply.
    """
    context_history = []
    # Add all messages from all preceding stages.
    for stage_num, messages in preceding_stages:
        context_history.extend(messages)
    
    # For the selected stage (which is complete) add only the assistant message (assumed to be the first message)
    assistant_msg = selected_stage[1][0]
    context_history.append(assistant_msg)
    
    return context_history

def format_conversation(message_list):
    """
    Formats a list of message dicts into a newline-separated string.
    """
    return "\n".join([f"{msg['timestamp']} - {msg['role']}: {msg['message']}" for msg in message_list])


# Example usage of the loop to generate training data.
if __name__ == "__main__":
    total_iterations = 200  # or any other number of iterations you need
    for count in range(total_iterations):
        print(f"\n===== Iteration {count+1} of {total_iterations} =====")
        try:
            # Get the conversation stages in order.
            sorted_stages = get_stages_in_order(conversation_history_stages)
            
            # Select a random complete stage (excluding the last stage) as the current stage.
            selected_stage, preceding_stages = select_random_complete_stage(sorted_stages)
            
            # For a complete stage, assume the target user message is the second message.
            user_message = selected_stage[1][1]
            
            # Build the conversation context: all stages before the selected stage plus the assistant message from the current stage.
            context_history = build_context_history(preceding_stages, selected_stage)
            
            # Format the context history into a single string.
            formatted_context = format_conversation(context_history)
            
            # Also, format the entire selected stage (both messages) for reference.
            formatted_selected_stage = format_conversation(selected_stage[1])
            
            # The final target user message we want the model to respond to:
            last_message = f"{user_message['timestamp']} - {user_message['role']}: {user_message['message']}"
            
            print("\n--- Conversation History (Context) ---")
            print(formatted_context)
            print("\n--- Target User Message ---")
            print(last_message)
            
            # Prepare the prompt message for the model.
            prompt_message = textwrap.dedent(f"""
                The conversation history is as follows:
                {formatted_context}
                
                The current stage conversation is:
                {formatted_selected_stage}
                
                Please respond to the user's last message in the current stage, which is:
                {last_message}
            """)
            
            # Call the model (assumes teacher_model, client, and system_prompt are predefined)
            completion = client.chat.completions.create(
                model=teacher_model,
                messages=[
                    {'role': "system", 'content': system_prompt},
                    {'role': "user", 'content': prompt_message}
                ],
                reasoning_effort="low",
                store=True,
                metadata={
                    "model": "o3-mini-2025-01-31",
                    "test": 'A001'
                }
            )
            
            model_response = completion.choices[0].message.content
            print("\n=== Model Response ===")
            print(model_response)
            
        except Exception as e:
            print(f"An error occurred during iteration {count+1}: {e}")


# **Fine-Tuning Process**

We now transition to the OpenAI Dashboard, where we create a training dataset from our logs. The dataset includes metadata with the key `test` and the value `A001`. Afterward, we distill a 4.1-mini model.

---

# **Testing:** Fine-Tuned Model

In [ ]:
conversation_history = [
    {
        "timestamp": "8:07 PM",
        "role": "assistant",
        "content": "Hello! How can I help you today?"
    },
    {
        "timestamp": "8:07 PM",
        "role": "user",
        "content": "hello i need to purchase a vehicle"
    },
    {
        "timestamp": "8:07 PM",
        "role": "assistant",
        "content": "I'd be happy to help you find the perfect vehicle. Could you please tell me more about your needs? For example, which features or models interest you, and do you have a budget in mind?"
    },
    {
        "timestamp": "8:07 PM",
        "role": "user",
        "content": "i need something that can accommodate my family of 6 under 40k budget"
    },
    {
        "timestamp": "8:08 PM",
        "role": "assistant",
        "content": "Please wait while I search our inventory. I found a 2019 Nissan Pathfinder priced at $32,000 that may be ideal for your family. Would you like more details or to schedule a test drive?"
    },
    {
        "timestamp": "8:08 PM",
        "role": "user",
        "content": "ok i'll like to schedule a test drive"
    },
    {
        "timestamp": "8:08 PM",
        "role": "assistant",
        "content": "Great! To schedule the test drive, could you please provide your full name, a valid phone number, and your email address?"
    },
    {
        "timestamp": "8:08 PM",
        "role": "user",
        "content": "alan jordan, 6787011214, ajon@gmail.com"
    }
]

completion = client.chat.completions.create(
    model="ft:gpt-4.1-mini-2025-04-14:personal:a001:(obscured)",
    messages=conversation_history
    )

# Print the response from the model
response = completion.choices[0].message.content
print("Model Response:")
print(response)
